In [1]:
# ==============================================================================
# SCRIPT 1: DOWNLOAD AND UPLOAD DEPENDENCY SOURCE CODE (WITH TIMEOUTS)
# (Run this in an environment WITH internet access, like Colab Pro+)
# ==============================================================================
import os
import subprocess

# --- CONFIGURATION ---
GCS_BUCKET_NAME = "shhs-sleepedfx-colab-deps"
# -------------------

# 1. Authenticate to Google Cloud
from google.colab import auth
auth.authenticate_user()
print("✅ Authenticated to Google Cloud.")

# 2. Clear the old, incompatible packages from the GCS bucket
gcs_packages_path = f"gs://{GCS_BUCKET_NAME}/packages/"
print(f"Clearing old packages from {gcs_packages_path}...")
# The -f flag forces the deletion without prompting for confirmation
!gsutil -m rm -f -r {gcs_packages_path}*
print("✅ Old packages cleared.")

# 3. Create the requirements.txt file locally
# We add 'build' and 'wheel' which are necessary for compiling from source
requirements_content = """
build
wheel
pytorch-lightning
timm
pandas==2.2.2
pyarrow==19.0.0
gcsfs
mne
scikit-image
matplotlib
gradio
ray[default]
"""
with open("requirements.txt", "w") as f:
    f.write(requirements_content)

print("✅ requirements.txt created.")

# 4. Create a local directory to store the downloaded source code
local_source_dir = "/tmp/pip-sources"
os.makedirs(local_source_dir, exist_ok=True)
print(f"Created local directory: {local_source_dir}")

# 5. Use pip to download the SOURCE DISTRIBUTIONS for all packages, one by one with a timeout
print("🚀 Starting download of all required package source code...")

# Read the list of packages from the requirements file
with open("requirements.txt", "r") as f:
    packages = [line.strip() for line in f if line.strip()]

successful_downloads = 0
failed_downloads = []
# Loop through each package and download it individually
for package in packages:
    print(f"  -> Downloading {package}...")
    command = [
        "pip", "download",
        "--no-binary=:all:",
        "-d", local_source_dir,
        package
    ]
    try:
        # Run the download command with a 15-minute timeout (900 seconds)
        subprocess.run(command, timeout=900, check=True, capture_output=True, text=True)
        print(f"     ✅ Successfully downloaded {package}")
        successful_downloads += 1
    except subprocess.TimeoutExpired:
        print(f"     ⚠️ WARNING: Timed out downloading {package}. Skipping.")
        failed_downloads.append(package)
    except subprocess.CalledProcessError as e:
        print(f"     ❌ ERROR: Failed to download {package}. Error: {e.stderr}. Skipping.")
        failed_downloads.append(package)

print(f"\n✅ Download process complete. {successful_downloads} packages downloaded successfully.")
if failed_downloads:
    print("\n--- Failed Downloads ---")
    for pkg in failed_downloads:
        print(f"- {pkg}")
    print("------------------------")


# 6. Upload the successfully downloaded source code and requirements file to your GCS bucket
print(f"🚀 Uploading contents of {local_source_dir} to {gcs_packages_path}...")
!gsutil -m cp -r {local_source_dir}/* {gcs_packages_path}
!gsutil cp requirements.txt gs://{GCS_BUCKET_NAME}/

print("\n✅ All available source code files have been successfully uploaded to your GCS bucket.")
print("You can now run the training script in your Colab Enterprise environment.")

✅ Authenticated to Google Cloud.
Clearing old packages from gs://shhs-sleepedfx-colab-deps/packages/...
CommandException: 1 files/objects could not be removed.
✅ Old packages cleared.
✅ requirements.txt created.
Created local directory: /tmp/pip-sources
🚀 Starting download of all required package source code...
  -> Downloading build...
     ✅ Successfully downloaded build
  -> Downloading wheel...
     ✅ Successfully downloaded wheel
  -> Downloading pytorch-lightning...
     ⚠️ WARNING: Timed out downloading pytorch-lightning. Skipping.
  -> Downloading timm...
     ❌ ERROR: Failed to download timm. Error: ERROR: Cannot install timm==0.1.1, timm==0.1.10, timm==0.1.12, timm==0.1.14, timm==0.1.16, timm==0.1.18, timm==0.1.2, timm==0.1.20, timm==0.1.22, timm==0.1.24, timm==0.1.26, timm==0.1.28, timm==0.1.30, timm==0.1.4, timm==0.1.6, timm==0.1.8, timm==0.2.1, timm==0.3.1, timm==0.3.2, timm==0.3.3, timm==0.3.4, timm==0.4.12, timm==0.4.5, timm==0.4.9, timm==0.5.4, timm==0.6.11, timm==0.6.1